# Plotting Relational Networks and Deriving Relations

This notebook contains tutorials for some example uses of the relation derivation tables described in Raemaekers (et al.?, the future). <br><br>

Example uses are:

<li>
    
</li>


In [1]:
# %% SETUP - Import dependencies

#!pip install ipywidgets --upgrade

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch
from IPython.core.debugger import set_trace
#!pip install bottleneck --upgrade # Update bottleneck (np.array functions) avoid pandas throwing an error
import pandas as pd
import string
import networkx as nx # for graph networks
from derivationTablesFromSourceRelations import derivationTablesFromSourceRelations # Custom functions to create derivation tables
from deriveRelationsFromBaseline import deriveRelationsFromBaseline # Custom functions to derive relations from source


# Combinatorial entailment of comparative relations 

For "simple" (cf. infra for more complex ones) comparative relations like *more than* or *less than* (but technically also: *before* and *after*), we only need to specify three points.  

In [2]:
# {"tags": ["hide_input"]}

# For three defined points, identify the relation between them and then derive relation between A and C
def f(A,B,C):
    if A >= B: 
        if A == B:
            AxB = 'The same as'
        else:
            AxB = 'More than'
    else: AxB = 'Less than'
    if B >= C: 
        if B == C:
            BxC = 'The same as'
        else:
            BxC = 'More than'
    else: BxC = 'Less than'
    
    # Syllogism 
    syl = 'A is ' + AxB.lower() + ' B and ' + 'B is ' + BxC.lower() + ' C.'
    
    # Find derived relation using function
    relations = dict({'The same as': 0,
                     'More than': 1,
                     'Less than': 2})
    
    mutual, combi = derivationTablesFromSourceRelations(relations)
    r1 = list.index(list(relations.keys()), AxB)
    r2 = list.index(list(relations.keys()), BxC)

    # For now assuming 'linear'protocol by default, need not be the case necessarily...
    r3 = combi['Linear'][r1, r2]
    
    AxC = 'Therefore, A is ' + list(relations.keys())[r3].lower() + ' C.'
    return syl, AxC

print('Use these sliders to set the value of A, B and C. The system will tell you the relations between them, and derive A-C')
interact(f, A=(0, 10), B = (0,10), C = (0, 10));

Use these sliders to set the value of A, B and C. The system will tell you the relations between them, and derive A-C


interactive(children=(IntSlider(value=5, description='A', max=10), IntSlider(value=5, description='B', max=10)…

For "more complex" relations that involve sets or some range (e.g., time, quantities), one is really relating relations (between sets of points). 
For example, the relations "A during B" (or its inverse "A contains B"), specify how an event A (or B) is located within the outer boundaries of some range B (or A). Relations like "A overlaps with B" (or is overlapped by, as per Allen, 1983), "A meets B" (or "is met by", "starts", "is started by", "larger than or the same as") specify the relation between two sets of points (A1,A2) and (B1, B2)

In [3]:
# Specify three sets of points and plot three lines on a grid + derived relations

# Compute the relation between the sets AxB and BxC, to then derive AxC
# Before-less/after-more, During/contains, overlaps/overlapped by, meets/met by, starts/started by, finishes/finished by
# 
# How to do this efficiently? rank points in a list, then check order of A1 A2 B1 B2
    
   

# Add a plot to visualize relations between three lines
def plot_lines(A1, A2, B1, B2, C1, C2):
    
    # Clear previous plot
    plt.clf()
    
    # Plot each line with user-provided x-coordinates and fixed y-
    plt.plot([A1, A2], [7,7], 'r-', label='Line A')  # Line A in red at y=7
    plt.plot([B1, B2], [5,5], 'g-', label='Line B')  # Line B in green at y=5
    plt.plot([C1, C2], [3,3], 'b-', label='Line C')  # Line C in blue at y=3
    
    # Set plot properties
    plt.xlim(0, 10)
    plt.ylim(0, 10)
    plt.xlabel("X-axis")
    plt.ylabel("Y-axis")
    plt.title("Interactive Line Plot")
    plt.legend()
    plt.grid(True)
    plt.show()
    
    # Describe relations from input and compute derived relation A-C
    # Assume L-R, small-large input?
    # Find A-B relation
    if A1 < B1: # Assuming B2>B1 -> A1 < B2
        if A2 < B1: AxB = 'A before / smaller than B'
        elif A2 == B1: AxB = 'A meets B'
        elif A2 > B1: 
            if A2 > B2:  AxB = 'A contains B'
            elif A2 == B2:  AxB = 'A is finished by B'
            else:  AxB = 'A overlaps B'
    elif A1 == B1: # Special case where A1 == B2 not considered, as it implies B is a point
        if A2 < B2:  AxB = 'A starts B'
        elif A2 == B2:  AxB = 'A = B'
        elif A2 > B2:  AxB = 'A is started by B'
    else: #A1 > B1
        if A1 > B2:  AxB = 'A after/more than B'
        elif A1 == B2:  AxB = 'A is met by B // All A larger than or equal to B'
        elif A1 < B2:  AxB = 'A is overlapped by B'
        if A2 < B2:  AxB = 'A during/part of B'
        elif A2 == B2:  AxB = 'A finishes B'
        elif A2 > B2:  AxB = 'A is overlapped by B'
    # Do the same for B-C
    if B1 < C1: # Assuming B2>B1 -> A1 < B2
        if B2 < C1: BxC = 'B before / smaller than C'
        elif B2 == C1: BxC = 'B meets C'
        elif B2 > C1: 
            if B2 > C2:  BxC = 'B contains C'
            elif B2 == C2:  BxC = 'B is finished by C'
            else:  BxC = 'B overlaps C'
    elif B1 == C1: # Special case where A1 == B2 not considered, as it implies B is a point
        if B2 < C2:  BxC = 'B starts C'
        elif B2 == C2:  BxC = 'B = C'
        elif B2 > C2:  BxC = 'B is started by C'
    else: #A1 > B1
        if B1 > C2:  BxC = 'B after/more than C'
        elif B1 == C2:  BxC = 'B is met by C // All B larger than or equal to C'
        elif B1 < C2:  BxC = 'B is overlapped by C'
        if B2 < C2:  BxC = 'B during/part of C'
        elif B2 == C2:  BxC = 'B finishes C'
        elif B2 > C2:  BxC = 'B is overlapped by C'
    
    
    # Then find derived relation using derivation tables (need to expand tables first)
    
    return AxB, BxC
    
# Create sliders
interact(plot_lines, A1=(0, 10), A2=(0, 10), B1 = (0,10), B2 = (0,10), C1 = (0, 10), C2 = (0, 10));


# Output derived relation(s)

interactive(children=(IntSlider(value=5, description='A1', max=10), IntSlider(value=5, description='A2', max=1…

We can also do this the other way around - user provides relations between A-B and B-C (or other protocol) and function returns derived relations

In [ ]:
def plot_rels(AxB, BxC): # Given two relations specified by user, plot an example scenario and compute derived relations
    
    ## Could specify relations based on input, but issues with assymetrical derivations?
    #relations = dict() #init
    #for r in range(len([AxB, BxC])): 
    #    if not [AxB, BxC][r] in relations.keys():
    #        relations[[AxB, BxC][r]] = r    
    #    else: continue
    #Alternatively use default
    relations = dict({'Same as': 0,
             'Different from': 1,
             'Opposite to': 2,
             'More than': 3,
             'Less than': 4,
             'Contains': 5,
             'Is part of': 6,
             'Before': 7,
             'After': 8})
    
    
    r1 = list.index(list(relations.keys()), AxB)
    r2 = list.index(list(relations.keys()), BxC)

    # Create derivation tables
    mutual, combi = derivationTablesFromSourceRelations() # Default relations as input for now

    
    # Find derived relations
    mut1 = mutual[list(relations.values())[r1]]    
    BxA = 'B is ' + list(relations.keys())[mut1].lower() + ' A'
    mut2 = mutual[list(relations.values())[r2]]
    CxB = 'C is ' + list(relations.keys())[mut2].lower() + ' B'    
    com = combi['Linear'][list(relations.values())[r1], list(relations.values())[r2]] # Again assuming linear protocol
    if com >=0: # Only compute mutual combinatorial entailment if well-defined
        mutcom = mutual[com]
        AxC = 'A is ' + list(relations.keys())[com].lower() + ' C'
        CxA = 'C is ' + list(relations.keys())[mutcom].lower() + ' A'

    else: 
        AxC = 'I do not have enough info to derive A-C'
        CxA = ' or to derive C-A'
    return BxA, CxB, AxC, CxA

# Create drop-down menu to select relations
interact(plot_rels, AxB=['Same as','Different from', 'Opposite to', 'More than', 'Less than', 'Contains', 'Is part of', 'Before', 'After'],
            BxC=['Same as','Different from', 'Opposite to', 'More than', 'Less than', 'Contains', 'Is part of', 'Before', 'After']);


# Visualize Relational Network
Can also use derivation function output to plot the relational network (baseline and derived relations).
Provided a set of user-defined baseline relations, use custom functions based on derivation tables to derive relations, then plot the full network of baseline and derived relations as a graph network (though simply on a cartesian plot). 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pdb
import string
from matplotlib.patches import FancyArrowPatch

from derivationTablesFromSourceRelations import derivationTablesFromSourceRelations
from deriveRelationsFromBaseline import deriveRelationsFromBaseline

# plot parameteres
radius = .2 # Determines curvature of lines between stimuli, can tweak to make plot more readable
# Between .15 and .3 seems to provide best results
relColor = 'black'
drelColor = 'grey'
protocol = 'Linear' # For one-to-many or many-to-one, the 'one' is plotted in the middle
title = 'Baseline Trained Relational Network and Tested Derived Relations'
# arrowStyle = "simple, head_length=50, head_width=15, tail_width=5" # Simple arrow growing thinner
arrowStyle = "fancy, head_length=50, head_width=15, tail_width=5" # Pointed arrow growing thinner
label_offset = .75 # Play around with how close labels are plotted to lines
relLabelFontSize = 12
sLabelFontSize = 20
sDotSize = 40


# Define default relations
relations = dict({'Same as': 0,
             'Different from': 1,
             'Opposite to': 2,
             'More than': 3,
             'Less than': 4,
             'Contains': 5,
             'Is part of': 6,
             'Before': 7,
             'After': 8})
mutual, combi = derivationTablesFromSourceRelations(relations)
# Deinfe baseline network
baseline = dict({'Less than': [(0,1)],
                   'More than': [(1,2)], 
              # 'Opposite to': [(0,3), (0,6)]
             })
# derive relations (or do on the spot while plotting?)
plot = False
printRels = True
unique = []
for rels in baseline.keys():
    for rel in baseline[rels]: 
        for s in rel: 
            if s not in unique: unique.append(s)
n_stim = len(unique) 
sLabs = ['A', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3', 'N']
relTab, derived = deriveRelationsFromBaseline(baseline, plot, printRels, n_stim, sLabs)


vertices = []
def polygon_coordinates(n_nodes, radius=n_stim*45, center=(n_stim*50, n_stim*50)):
    # Calculate angle between vertices
    if n_nodes > 3: angle = 2 * np.pi / (n_nodes-1)
    else: angle = 2 * np.pi / (n_nodes)
        
    # Compute the vertices
    vertices = []
    if protocol != 'Linear': # Plot network around the 'One' (center)for OTM/MTO
        vertices.append((center))
        for i in range(n_nodes-1): # skip one for center S
            x = center[0] + radius * np.cos(i * angle) # Calculate polygon coords around center
            y = center[1] + radius * np.sin(i * angle)
            vertices.append((x, y))
    else:
        for i in range(n_nodes):
            x = center[0] + radius * np.cos(i * angle) 
            y = center[1] + radius * np.sin(i * angle)
            vertices.append((x, y))
    return vertices

vertices = polygon_coordinates(n_stim)


# Create a Cartesian grid plot without showing the grid
plt.figure(figsize=(20, 20))  # Set figure size
plt.plot([], [])  # Create an empty plot (optional for axes setup)
plt.xlim(0,  n_stim*100)  # Set x-axis range
plt.ylim(0,  n_stim*100)  # Set y-axis range


# Intitialize to keep track of plotted network
plottedS = dict()
plottedRels = dict()


# Loop through baseline relations
for rels in baseline.keys():
    for rel in baseline[rels]:
        for s in range(2):
            # First plot dot if new stimulus
            if not sLabs[rel[s]] in plottedS.keys():
                plottedS[sLabs[rel[s]]] = [vertices[len(plottedS.keys())]] 
                # plot a labeled point on the grid
                x = plottedS[sLabs[rel[s]]][0][0]
                y = plottedS[sLabs[rel[s]]][0][1]
                plt.plot(x, y, 'o', markersize = sDotSize, color = 'grey')  # Plot point
                plt.text(x, y , sLabs[rel[s]], 
                         fontweight = 'bold', fontsize=sLabelFontSize, ha='center', va = 'center')  # Label next to the point
        # get coords for relation
        x_start, y_start = plottedS[sLabs[rel[0]]][0][0], plottedS[sLabs[rel[0]]][0][1]        
        x_end, y_end = plottedS[sLabs[rel[1]]][0][0], plottedS[sLabs[rel[1]]][0][1]
        # Plot a curved line using FancyArrowPatch 
        # pdb.set_trace()
        baselineArrow = FancyArrowPatch((x_start, y_start), (x_end, y_end),
                                connectionstyle="arc3,rad={}".format(radius),  # Controls the curvature
                                arrowstyle= arrowStyle, color=relColor, linewidth=1.5)
        plt.gca().add_patch(baselineArrow)
        # Label the curve, position depending on symmetry of relation
        mid_x, mid_y = (x_start + x_end)/ 2, (y_start + y_end)/ 2# midpoint for positioning
        # make label position function of direction of arrow up/down-left/right
                
            
        dx, dy = x_end - x_start, y_end - y_start # Vector from start to end
        
        distance = np.sqrt(dx**2 + dy**2) # Calculate the distance between start and end points
        # Calculate angle of the line segment for consistent radial offset
        angle = np.arctan2(dy, dx) + np.pi / 2  # Rotate by 90 degrees to get perpendicular

        offset = radius * distance # Offset based on radius
        # Azimuth point for label, shifted by label_offset in the radial direction
        azimuth_x = mid_x - offset * np.cos(angle) *label_offset
        azimuth_y = mid_y - offset * np.sin(angle) *label_offset
        plt.text(azimuth_x, azimuth_y, rels, color=relColor, 
                 fontsize=sLabelFontSize, ha='center', fontweight = 'bold')
        
plotted = []
# Loop through derived relations
for drels in derived.keys():
    for drel in derived[drels]:
        # get coords for relation
        x_start, y_start = plottedS[sLabs[drel[0]]][0][0], plottedS[sLabs[drel[0]]][0][1]        
        x_end, y_end = plottedS[sLabs[drel[1]]][0][0], plottedS[sLabs[drel[1]]][0][1]
        if x_start != x_end and y_start != y_end: # no self relations
            if drel not in plotted: # No duplicates (better to filter in derivation script!!)
                # Plot a curved line using FancyArrowPatch 
                derivedArrow = FancyArrowPatch((x_start, y_start), (x_end, y_end),
                                        connectionstyle="arc3,rad={}".format(radius),  # Controls the curvature
                                        arrowstyle=arrowStyle, color=drelColor, 
                                        linestyle = ':', linewidth=1.5)
                plt.gca().add_patch(derivedArrow)
                # Label the curve, position depending on symmetry of relation
                mid_x, mid_y = (x_start + x_end)/ 2, (y_start + y_end)/ 2# midpoint for positioning
                # make label position function of direction of arrow up/down-left/right
                        
                    
                dx, dy = x_end - x_start, y_end - y_start # Vector from start to end
                
                distance = np.sqrt(dx**2 + dy**2) # Calculate the distance between start and end points
                # Calculate angle of the line segment for consistent radial offset
                angle = np.arctan2(dy, dx) + np.pi / 2  # Rotate by 90 degrees to get perpendicular
        
                offset = radius * distance # Offset based on radius
                # Azimuth point for label, shifted by label_offset in the radial direction
                azimuth_x = mid_x - offset * np.cos(angle) *label_offset
                azimuth_y = mid_y - offset * np.sin(angle) *label_offset
                plt.text(azimuth_x, azimuth_y, drels, color=drelColor, 
                         fontsize=relLabelFontSize, ha='center', fontweight = 'bold')
                plotted.append((drel))


plt.title(title, fontsize=24, fontweight = 'bold')

# Ensure grid is not displayed
plt.grid(False)
plt.xticks([])  # Removes x-axis ticks
plt.yticks([])  # Removes y-axis ticks

# Hide spines (the axis lines)
for spine in plt.gca().spines.values():
    spine.set_visible(False)
# Display the plot
plt.show()

In [ ]:
# Alternative using networxx to plot graph network (requires (a lot) more memory, but likely better programmed :) )


# Initialize the graph
G = nx.DiGraph()  # Directed graph for relationships

# Define relations and stimuli
relations = {
    "Same as": 0,
    "Different from": 1,
    "Opposite to": 2,
    "More than": 3,
    "Less than": 4,
    "Contains": 5,
    "Is part of": 6,
    "Before": 7,
    "After": 8,
}
# Optional to have relation-specific colors (get better colors tho)
relColors = False
if relColors:
    rel_colors = {
        "Less than": "blue",
        "More than": "green",
        "Same as": "yellow",
        "Different from": "red",
        "Opposite to": "yellow",
        "Contains": "orange",
        "Is part of": "brown",
        "Before": "cyan",
        "After": "pink",
    }

# Define baseline network
baseline = {
    "Same as": [(0, 1), (0, 4)],
    "More than": [(0, 2), (0, 5)],
    "Less than": [(0, 3), (0, 6)]
}
plot = False
printRels = False
unique = []
for rels in baseline.keys():
    for rel in baseline[rels]: 
        for s in rel: 
            if s not in unique: unique.append(s)
n_stim = len(unique) 
sLabs = ["A", "B1", "B2", "B3", "C1", "C2", "C3", "N"]
relTab, derived = deriveRelationsFromBaseline(baseline, plot, printRels, n_stim, sLabs)

# Define labels for stimuli
label_offset = 1
edge_colors = []
edge_styles = []
# Add nodes and edges for baseline relations
for rel, pairs in baseline.items():
    for src, tgt in pairs:
        G.add_node(sLabs[src])  # Add source node
        G.add_node(sLabs[tgt])  # Add target node
        G.add_edge(sLabs[src], sLabs[tgt], relation=rel)  # Add edge with relation label
        if relColors:
            edge_colors.append(rel_colors[rel])
        else:
            edge_colors.append(relColor)
        edge_styles.append("-")

for rel, pairs in derived.items():
    for src, tgt in pairs:
        G.add_node(sLabs[src])  # Add source node
        G.add_node(sLabs[tgt])  # Add target node
        G.add_edge(sLabs[src], sLabs[tgt], relation=rel)  # Add edge with relation label
        if relColors:
            edge_colors.append(rel_colors[rel])
            
        else:
            edge_colors.append(drelColor)
        edge_styles.append(":")
# Use a circular layout for node positions
pos = nx.circular_layout(G)

# Plot the graph
plt.figure(figsize=(12, 12), dpi=50)  # Lower DPI to reduce pixel dimensions

# Draw nodes with labels
nx.draw_networkx_nodes(G, pos, node_size=800, node_color="lightgrey")
nx.draw_networkx_labels(G, pos, font_size=14, font_weight="bold")



# # Draw edges with labels
nx.draw_networkx_edges(G, pos, arrowstyle="simple, tail_width = .1, head_width = .4, head_length = 1",
                        style = edge_styles, arrowsize=20, edge_color=edge_colors, connectionstyle="arc3,rad=0.3")
edge_labels = nx.get_edge_attributes(G, "relation")
# nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='black', font_size=12)

# Alternatively draw labels on the curves, to adress assymetrical relations
i = 0
for (src, tgt, data) in G.edges(data=True):
    i +=1 
    # Get source and target positions
    x1, y1 = pos[src]
    x2, y2 = pos[tgt]

    # Calculate midpoint
    mx, my = (x1 + x2) / 2, (y1 + y2) / 2
    mid_x, mid_y = (x_start + x_end)/ 2, (y_start + y_end)/ 2# midpoint for positioning
    # make label position function of direction of arrow up/down-left/right       
    dx, dy = x_end - x_start, y_end - y_start # Vector from start to end
    distance = np.sqrt(dx**2 + dy**2) # Calculate the distance between start and end points
    # Calculate angle of the line segment for consistent radial offset
    angle = np.arctan2(dy, dx) + np.pi / 2  # Rotate by 90 degrees to get perpendicular
    offset = radius * distance # Offset for curve/label based on radius
    # Azimuth point for label, shifted by label_offset in the radial direction
    azimuth_x = mid_x - offset * np.cos(angle) *label_offset
    azimuth_y = mid_y - offset * np.sin(angle) *label_offset
    # Draw the label
    plt.text(
        azimuth_x,
        azimuth_y,
        data["relation"],
        fontsize=12,
        color=edge_colors[i],
        ha="center",
        va="center",
    )

# Set plot title
plt.title("Relational Network Graph", fontsize=24, fontweight="bold")

# Hide axes
plt.axis("off")

# Show the plot
plt.show()
